In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

from PIL import Image

Datasets: [here](https://paperswithcode.com/datasets?q=&v=lst&o=match&mod=images&task=image-super-resolution)


## Sources
- https://openmmlab.medium.com/awesome-datasets-for-super-resolution-introduction-and-pre-processing-55f8501f8b18
- https://paperswithcode.com/dataset/imagenet
- https://paperswithcode.com/dataset/div2k
- https://github.com/sgrvinod/Deep-Tutorials-for-PyTorch


## mindmap
- GAN - one NN for superres and second one for img rating in zero-sum game




In [ ]:
class Div2kDataset(Dataset):
    def __init__(self, dir='./data', transform=None):
        self.dir = dir
        self.transform = transform
        self.data = sorted([x for x in os.listdir(self.dir)])
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(f'{self.dir}/{self.data[idx]}')
        return img
        # return torch.tensor(np.array(img), dtype=torch.float)

train_dataset_hd = Div2kDataset('DIV2K/HR')
train_dataset_lr = Div2kDataset('DIV2K/4xLR')

display(train_dataset_lr[0])
display(train_dataset_hd[0])